In [7]:
from bs4 import BeautifulSoup
from bson.objectid import ObjectId
import requests
import pandas as pd
import numpy as np
import os
from PIL import Image
import os

In [8]:

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os

db_uri = os.getenv('DB_URL')
client = MongoClient(db_uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [9]:
database = client.get_default_database('estate-inf')
try:
    database.create_collection('estates')
except:
    get_data = database.get_collection('estates')
get_data = database.get_collection('estates')

In [10]:
def get_title(soup):
    try:
        title = soup.find("h1", attrs={"class":'title-product'})
        title_value = title.text
        title_string = title_value.strip().split('\n')
        
        if title_string == ['']:
            return 'Không xác định'
        return title_string[0]
    except:
        return 'Không xác định'
    
def get_info(soup):
    try:
        array_1 = soup.find_all("ul", attrs={'class':'list-attr-hot clearfix'})[0].text.strip().split('\n')
        array_2 = soup.find_all("ul", attrs={'class':'list-attr-hot clearfix'})[1].text.strip().split('\n')
        
        if(array_2[0] != ''):
            filtered_array_2 = [item for item in array_2 if item]
        else: filtered_array_2 = []
        
        filtered_array_1 = [item for item in array_1 if item]
        
        
        if len(filtered_array_2) == 2:
            return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], filtered_array_2[1], 0, 0]
        elif len(filtered_array_2) == 4:
            return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], filtered_array_2[1], filtered_array_2[3], 0]
        elif len(filtered_array_2) == 6:
            return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], filtered_array_2[1], filtered_array_2[3], filtered_array_2[5]]
        return [filtered_array_1[1], filtered_array_1[3], filtered_array_1[5], 0, 0, 0]
    except:
        return 0

def get_location(soup):
    try:
        location = soup.find("ul", attrs={'class':'breadcrumb clearfix'}).text.strip().split('\n')
        if location != ['']:
            return [location[1], location[0]]
        return ['Không xác định', 'Không xác định']
    except:
        return 0   

def get_illustrate(soup):
    try:
        array = soup.find("div", attrs={'class':'ct-pr-sum'}).text.strip().split('\n')
        if 'Tìm kiếm theo từ khóa:' in array:
            index = array.index('Tìm kiếm theo từ khóa:')
            array = array[:index]

        return ' '.join(array)
    except:
        return ''

def get_image(soup, id):
    try:
        img_url = soup.find("img", attrs={'class':'img-product-face'})
        if(img_url):
            img_url = 'https://bds.com.vn/images/noimage.png'
        else:
            img_url = soup.find_all("img")[1]['src']
            
        
        idx = img_url.rfind('.')
        img_name = f'img{id}' + img_url[idx:]
        img_response = requests.get(img_url, stream=True)   

        try:
            with open('img/'+img_name, 'wb') as f:
                f.write(img_response.content)
                img = Image.open('img/'+ img_name)
                img_resized = img.resize((250, 250))
                img_resized.save('resized_images/' + img_name)
        except: 
            return 0
        return img_name
    except: 
        return 0
    
def get_phoneNumber(soup):
    try:
        raw = soup.find("a", attrs={'class':'call'})['href']
        idx = raw.rfind(':')
        if raw[idx+1:] == '':
            return 0
        return raw[idx+1:]
    except:
        return 0


In [11]:
id = 1
data = []
pages = 20

In [ ]:

if __name__ == '__main__':
    URLs = ['https://bds.com.vn/mua-ban-nha-dat', 'https://bds.com.vn/mua-ban-dat', 'https://bds.com.vn/cho-thue-nha-dat']
    
    for idx, URL in enumerate(URLs): 
        for i in range(1, pages):
            webpage = requests.get(URL)
            soup = BeautifulSoup(webpage.content, "html.parser")
            links = soup.find_all("a", attrs={'class':'title-item-nhadat'})

            links_list = []

            for link in links:
                    links_list.append(link.get('href'))
            new_id = ObjectId()
            for link in links_list:
                print(link)
                d = {
                    "title": str,
                    "location": str,
                    "price": str,
                    "area": str,
                    "date": str,
                    "image": str,
                    "illustrate": str,
                    "types": str,
                    "option": str,
                    "phone_number": str,
                    "num_bedroom": str,
                    "num_wc": 0,
                    "num_floor": 0,
                }
                new_webpage = requests.get(link)
                new_soup = BeautifulSoup(new_webpage.content, "html.parser")
                
                info = get_info(new_soup)
                location = get_location(new_soup)
                phone_number = get_phoneNumber(new_soup)
                image = get_image(new_soup, id)
                
                if phone_number == 0: continue
                if info == 0: continue
                if info[0] == 'Diện tích :': continue
                if image == 0: continue
                if location == 0: continue
                
                if idx == 0:
                    d['types'] = 'Mua bán nhà đất'
                elif idx == 1:
                    d['types'] = 'Mua bán đất'
                else:
                    d['types'] = 'Cho thuê nhà đất'
                
                d['title'] = get_title(new_soup)
                d['location'] = location[0]
                d['price'], d['area'], d['date'] = info[0], info[1], info[2]
                d['image'] = image
                d['illustrate'] = get_illustrate(new_soup)
                d['option'] = location[1]
                d['phone_number'] = phone_number
                d['num_bedroom'] = int(info[3])
                d['num_wc'] = int(info[4])
                d['num_floor'] = int(info[5])
                
                get_data.insert_one(
                    {
                        '_id': ObjectId(), 
                        "title": d['title'],
                        "location": location[0],
                        "price": info[0],
                        "area": info[1],
                        "date": info[2],
                        "image": image,
                        "illustrate": d['illustrate'],
                        "types": d['types'],
                        "option": location[1],
                        "phone_number": phone_number,
                        "num_bedroom": int(info[3]),
                        "num_wc": int(info[4]),
                        "num_floor": int(info[5]),
                    }
                )
                
                data.append(d)
                id += 1
            URL = URLs[idx] + f'-page{i}'


In [13]:
import json
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)